In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import math
import random

In [2]:
def phi(x, r):
    return r * x * (1 - x)

def get_window(r):
    return (max(0.0, (r - 1) / (2 * r)), (r + 1) / (2 * r))

In [3]:
def simple_iterations(epsilon, r):
    (left_border, right_border) = get_window(r)
    iterations = 0
    
    cur_x = random.uniform(left_border, right_border)
    next_x = phi(cur_x, r) 
    
    while (abs(cur_x - next_x) > epsilon):
        cur_x = next_x
        next_x = phi(cur_x, r)
        iterations += 1
        if (iterations > 1000):
            return (-100, 1000)
        
    return (next_x, iterations)

In [4]:
roots = []
roots.append(0)
roots.append(0)

for i in range(1, 10**4):
    cur_r = random.uniform(0.01, 0.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
print(roots)

[9999, 0]


In [5]:
roots = []
roots.append(0)
roots.append(0)

undefined_roots = []

for i in range(1, 10**4):
    cur_r = random.uniform(1.01, 2.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
    else:
        undefined_roots.append(cur_r)
print(roots) 
print(undefined_roots)

[0, 9972]
[2.9884389933972137, 2.9891840131178027, 2.9892874327180197, 2.9860662991990505, 2.9872373551603495, 2.9862315329806908, 2.985017439048111, 2.9878006253773948, 2.9897173118982625, 2.985950682441941, 2.988324706992932, 2.987961701376669, 2.9854551211590916, 2.986014288956162, 2.9874069315777914, 2.989380399144546, 2.988404091834407, 2.986868804259476, 2.9876614553421024, 2.9867879977530345, 2.989612653646445, 2.9870676494452333, 2.9865190777926154, 2.98975888292263, 2.986963477088587, 2.9861960885691907, 2.9890004616804333]


In [6]:
count = 0

for i in range(1, 10**4):
    cur_r = random.uniform(3.01, 10.0)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    if (answer < -50):
        count += 1
print(count)

9999


x = r * x * (1 - x)

phi(x) = r * x * (1 - x)

phi'(x) = r - 2rx

чтобы решение сходилось к корню, первое приближение должно лежать в окрестности корня, удовлетворяющей условию

|phi'(x)| <= 1

|phi'(x)| <= 1 => max(0.0, (r - 1) / (2 * r)) < x < (r + 1) / (2 * r)

Если r > 1, то 0 в этой области не содержится, и сходиться к нему не будет

x0 = 0

x1 = 1 - 1/r

Рассмотрим phi'(x0) = r

Чтобы в окрестности корня выполнялось |phi'(x)| < 1, должно быть |r| < 1 => r < 1

Значит, при 0 < r < 1 сходимость к x0 = 0 возможна

Рассмотрим phi'(x1) = 2 - r

|phi'(x)| < 1 => |2 - r| < 1 => 1 < r < 3

При 1 < r < 3 сходимость к x1 = 1 - 1/r возможна

Заметим, что области сходимости к x0 (0; 1) и к x1 (1; 3) не пересекаются

In [7]:
def simple_iterations_with_dir_detection(epsilon, r):
    (left_border, right_border) = get_window(r)
    points = []
    
    dir_changed = 0
    dir_saved = 0
    
    cur_x = random.uniform(left_border, right_border)
    next_x = phi(cur_x, r) 
    points.append(cur_x)
    points.append(next_x)
    iterations = 2
    
    
    while (abs(cur_x - next_x) > epsilon):
        cur_x = next_x
        next_x = phi(cur_x, r)
        points.append(next_x)
        iterations += 1
        if (iterations > 1000):
            return (-100, -100, -100)
        
    if (abs(next_x) > 1e-6 and abs(next_x - (1 - 1/r)) > 1e-6):
        return (-100, -100, next_x)
    root = next_x
    for i in range (0, len(points) - 1):
        cur_point = points[i]
        next_point = points[i + 1]
        d = (cur_point - root) * (next_point - root)
        if d > 0:
            dir_saved += 1
        elif d < 0:
            dir_changed += 1
        
    return (dir_saved, dir_changed, root)

In [10]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(0.01, 0.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
        errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

531007 3906
[]


In [11]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(1.01, 1.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer - (1 - 1 / cur_r)) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
                errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

528034 2424
[]


In [12]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(2.01, 2.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer - (1 - 1 / cur_r)) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
                errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

3277 579025
[(2.9872512493012278, -100), (2.9852844856962095, -100), (2.9850766436501432, -100), (2.988891960851868, -100), (2.9875557512148387, -100), (2.989048613781245, -100), (2.9895389544320725, -100), (2.98676368705719, -100), (2.985827464997501, -100), (2.9871913568694746, -100), (2.988751543660295, -100), (2.986412307016161, -100), (2.986116925863125, -100), (2.9871092124229124, -100), (2.9867173731728576, -100), (2.989842502943227, -100), (2.9889852911519843, -100), (2.989898352518742, -100), (2.9892131597178757, -100), (2.986999231696218, -100), (2.985499451751889, -100), (2.9893571734649313, -100), (2.9888507103712216, -100), (2.9859813182452055, -100), (2.9869917995837962, -100), (2.984991498624462, -100), (2.9885164756490727, -100), (2.9847916143631434, -100), (2.9897688795233797, -100), (2.9857631865453214, -100), (2.9879139224694686, -100), (2.989300538408587, -100), (2.9886765543350267, -100), (2.987565339181266, -100), (2.9888660241291727, -100), (2.9896100004251305, -

При 0 < r < 1 сходится в точку x0 = 0.

phi'(x0) = r => 0 < phi' < 1 => монотонная сходимость

При 1 < r < 2 сходится в точку x1 = 1 - 1/r

phi'(x1) = 2 - r => 0 < phi' < 1 => монотонная сходимость

При 2 < r < 3 сходится в точку x1 = 1 - 1/r

phi'(x1) = 2 - r => -1 < phi' < 0 => колебательная сходимость